In [1]:
from CharRNN import CharRNNV2
from Generation import *
from DataLoader import *
import torch
from onehotencoder import OneHotEncoder
from lightning.pytorch import Trainer
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
torch.set_float32_matmul_precision("high")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
endecode = OneHotEncoder()
vocab_size = OneHotEncoder.get_vocab_size(self = endecode)
num_layers = 3
n_gram = 1
dropped_out = 0.2
num_workers = 5
hidden_size = 1024
learning_rate = 5e-4
num_epochs = 200
batch_size = 128
temp = 1
p = 1

In [3]:
file_paths = [f'data/seqs_len{i}.txt' for i in range(18, 52)]
Data = Data(filepaths=file_paths,encoder=endecode,n_gram=n_gram,batch_size=batch_size,num_workers=num_workers,num_epochs=num_epochs)
train_loader, val_loader, total_steps, warmup_steps = Data.get_loaders()

In [ ]:
charRNN = CharRNNV2(vocab_size, num_layers, n_gram, total_steps, warmup_steps, learning_rate, hidden_size, dropped_out).to(device)
trainer = Trainer(
    max_epochs=200,
    accelerator="cuda",
    precision='16-mixed',
    gradient_clip_val=5.0,
    logger=TensorBoardLogger("tb_logs", name="char_rnn"),
    callbacks=[
        ModelCheckpoint(monitor="valid_loss", mode="min"),
        EarlyStopping(monitor="valid_loss", patience=5),
    ],
    profiler="pytorch",
)
tuner = Tuner(trainer)
lr_find_results = tuner.lr_find(
    charRNN,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    min_lr=1e-6,
    max_lr=1.0,
    early_stop_threshold=None,
    update_attr=True,
)
trainer.fit(charRNN, train_loader, val_loader)
torch.save(charRNN.state_dict(), "Models/charRNNv1-gram.pt")

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Finding best initial lr: 100%|██████████| 100/100 [00:01<00:00, 59.58it/s]
Learning rate set to 0.0002187761623949553
Restoring states from the checkpoint path at /home/abog/PycharmProjects/Drug-Discovery/.lr_find_12ace012-77b9-4f8a-93c5-01934b4ebc84.ckpt
Restored all states from the checkpoint at /home/abog/PycharmProjects/Drug-Discovery/.lr_find_12ace012-77b9-4f8a-93c5-01934b4ebc84.ckpt
FIT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA

In [ ]:
charRNN = torch.load('Models/charRNNv1-gram.pt').to(device)
filepath = 'data/GRUOnly1P1-gram.txt'
generator = Generator(charRNN, endecode, vocab_size, n_gram, p, temp)
generator.generate(filepath)